# SNAC API grab - related orgs

Decided to separate this from related people since I've finished the people import and am guessing that the orgs will require something very different.

In [1]:
import time
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import json

In [2]:
query = """
{
    "command": "read",
    "constellationid": 83834110
}
"""
url = 'https://api.snaccooperative.org/'

In [3]:
# query SNAC for Bond's constellation

response = requests.put(url, query)
print(response)

<Response [200]>


In [4]:
text = response.text
text_json = json.loads(text)

relations = text_json['constellation']['relations']

relations_data = pd.DataFrame([(i['targetConstellation'], i['content'], i['targetEntityType']['term']) for i in relations])
relations_data.columns = ['constellation_id', 'name', 'entity_type']
relations_data.head()

,constellation_id,name,entity_type
0,5532829,American Civil Liberties Union.,corporateBody
1,76111561,American Folklife Center,corporateBody
2,47287903,American Foundation For Equal Rights.,corporateBody
3,33162909,Antioch review.,corporateBody
4,47966548,Atlanta University Center (Ga.),corporateBody


In [5]:
relations_data.entity_type.value_counts()

entity_type
person           155
corporateBody     53
family             3
Name: count, dtype: int64

In [6]:
relations_data[relations_data.entity_type == 'family']

,constellation_id,name,entity_type
10,29280972,Bond family.,family
158,5965928,Rubinstein family.,family
205,70036600,"Wilson, Bill (William Bryan), 1914-1993",family


In [7]:
# think I don't necessarily want to deal with that right now. Bit weird though. Shoutout Bill Wilson.
orgs = relations_data[relations_data.entity_type == 'corporateBody']

In [10]:
orgs.name

0                        American Civil Liberties Union.
1                               American Folklife Center
2                  American Foundation For Equal Rights.
3                                        Antioch review.
4                        Atlanta University Center (Ga.)
5                                  Atlanta Urban League.
7                                     Barnes Foundation.
20                  Brotherhood of Sleeping Car Porters.
25                          Center for Community Change.
26     Central Michigan University. Office of the Pre...
31     Citizens Board of Inquiry into Health Services...
37                 Committee on Appeal for Human Rights.
38      Communist Party of the United States of America.
43                             David Pearson Associates,
47     Democratic National Convention (1968 : Chicago...
48     Democratic National Convention (1984 : San Fra...
49                                Democratic Party (Ga.)
50                             

In [11]:
# lotta weird stuff happening here (why three SEFs?)
# think I want to just scrape everything and then decide what I want to do with them.

In [13]:
orgs.loc[0,:]

constellation_id                            5532829
name                American Civil Liberties Union.
entity_type                           corporateBody
Name: 0, dtype: object

In [14]:
# test query: let's look at the ACLU's response

test_data = {
    "command": "read",
    "constellationid": "5532829"
}

test_query = json.dumps(test_data)
test_response = requests.put(url, test_query)
test_json = json.loads(test_response.text)

In [30]:
test_json['constellation']['nameEntries'][0]['original']

'American Civil Liberties Union'

In [16]:
# well I understand why it took so long this thing is freaking huge
# full history is too big to display on our site, we should stick to redirecting

org_data = []

for idx, row in orgs.iterrows():
    
    data = {
        "command": "read",
        "constellationid": row['constellation_id']
    }

    query = json.dumps(data)
    response = requests.put(url, query)
    json_text = json.loads(response.text)

    constellation = json_text['constellation']

    try:
        name = constellation['nameEntries'][0]['original']
    except:
        name = ''

    arkid = constellation['ark']

    info = {'name': name, 'arkid': arkid, 'constellation_id': row['constellation_id']}

    org_data.append(info)